<img src="https://octodex.github.com/images/labtocat.png" width=200 height=200 />

## Cannabis Genomics dataset
In this project, I'm using SQL within a Python environment to perform data mining⛏️. Data mining involves discovering patterns and insights from large datasets, using techniques from statistics, machine learning, and database systems. I'll be exploring the Cannabis genomics dataset available on Google's BigQuery public data.

My SQL proficiency is mainly acquired by the [intro to SQL kaggle course](https://www.kaggle.com/learn/intro-to-sql), although I did have some prior SQL knowledge. I'm gonna practice it even more with a hands-on attitude. This is exciting! ✨🤩

# Extracting data from BigQuery

Imports

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

Fetch and list the tables from the dataset

In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the dataset
dataset_ref = client.dataset("genomics_cannabis", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Print the tables in the dataset
tables=list(client.list_tables(dataset))
for table in tables: print(table.table_id) 

Using Kaggle's public dataset BigQuery integration.
MNPR01_201703
MNPR01_reference_201703
MNPR01_transcriptome_201703
cs10_gff
cs3k_project_info
cs3k_vcf_cs10_dv090
sample_info_201703


Pick a table to explore. For instance, Sample_info table

In [3]:
# API request - fetch the table with its reference
sample_info_table = client.get_table(
    dataset_ref.table("sample_info_201703"))

# Preview the first five lines of the table
client.list_rows(sample_info_table, max_results=5).to_dataframe()

,SRA_Sample_s,Sample_Name_s,cultivar_s,Library_Name_s
0,SRS266493,CS-USO31-DNA,None,CS-US_SIL-1
1,SRS266493,CS-USO31-DNA,None,CS-US_SIL-2
2,SRS266493,CS-USO31-DNA,None,CS-US_SIL-1
3,SRS266493,CS-USO31-DNA,None,CS-US_SIL-2
4,SRS266713,CS-FINOLA-DNA,None,CS-FN_SIL-1


> Sample_info contains fields extracted for each SRA sample, including the SRA sample ID and other data that give indications about the type of sample. Sample types include: strain, library prep methods, and sequencing technology. See SRP008673 for an example of upstream sample data. SRP008673 is the University of Toronto sequencing of Cannabis Sativa subspecies Purple Kush. 
[DATA CARD](https://www.kaggle.com/datasets/bigquery/genomics-cannabis/data)


To avoid running into the 5TB query limit for each kaggler per month, we can **estimate how much data the query will scan** before actually running it by creating a QueryJobConfig object. 

Print the data size the query will process

In [4]:
# Query to get all columns from every row where the SRA_Sample_s column has value "SRP008673"
query = """
        SELECT *
        FROM `bigquery-public-data.genomics_cannabis.sample_info_201703`
        WHERE SRA_Sample_s = 'SRP008673' 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 85578 bytes.


In [5]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

# Execute the query and fetch data into a DataFrame
client.query(query,safe_config).to_dataframe()

,SRA_Sample_s,Sample_Name_s,cultivar_s,Library_Name_s


That's weird. There's no such sample id. It was mentioned in the data card though.

# Exploratory Data Analysis

In [6]:
# Query to get a row count for each sample_id (originally named SRA_Sample_s column) 
query = """
        SELECT SRA_Sample_s AS sample_id ,COUNT(1) AS num_reads
        FROM `bigquery-public-data.genomics_cannabis.sample_info_201703`
        GROUP BY SRA_Sample_s 
        ORDER BY num_reads DESC
        """
client.query(query,safe_config).to_dataframe()

,sample_id,num_reads
0,SRS266795,32
1,SRS1357165,6
2,SRS1357179,6
3,SRS1598708,6
4,SRS1598706,6
...,...,...
1165,SRS1357173,1
1166,SRS1357172,1
1167,SRS1357169,1
1168,SRS374081,1


In [7]:
# Query to get a row count for each sample_id (originally named SRA_Sample_s column) 
query = """
        with num_table AS
        (SELECT SRA_Sample_s AS sample_id ,COUNT(1) AS num_reads
        FROM `bigquery-public-data.genomics_cannabis.sample_info_201703`
        GROUP BY SRA_Sample_s 
        ORDER BY num_reads DESC)
        SELECT num_reads, COUNT(1) AS counts
        FROM num_table
        GROUP BY num_reads
        ORDER BY num_reads
        """
df=client.query(query,safe_config).to_dataframe()
df

,num_reads,counts
0,1,1073
1,2,57
2,3,5
3,4,17
4,5,3
5,6,14
6,32,1


In [8]:
fig = px.pie(df, names='num_reads', values='counts', title="Distribution of reads' repetitions",
             color_discrete_sequence=sns.color_palette("Spectral",7).as_hex()[::-1])
fig.show()

There is an outlier with 32 reads' repetitions but most reads (91% of the reads) are only repeated once.

In [9]:
# Query to get a row count for each cultivar
query = """
        SELECT cultivar_s AS cultivar ,COUNT(1) AS num_reads
        FROM `bigquery-public-data.genomics_cannabis.sample_info_201703`
        GROUP BY cultivar 
        ORDER BY num_reads DESC
        """
client.query(query,safe_config).to_dataframe()

,cultivar,num_reads
0,None,182
1,cv. Santhica 27,72
2,Sour_Diesel,14
3,Cannatonic,9
4,AC/DC,8
...,...,...
844,Tyra_Banks_Conspiracy_Kush_#5,1
845,Kyle_Kushmans_Strawberry_Cough,1
846,Malanje_Local_90's_Liamba_Angola,1
847,Hybrid_Krasnodarskii_10_FB_Russia,1


According to this sample_info table, the majority of reads is of an unknown strain (cultivar = 'None'). Cannabis sativa 
[(cv. Santhica 27)](https://www.hemp-it.coop/en/produit/hemp-santhica-27/)
is the most frequently sequenced cultivar.  

<img src="https://www.iqc.co.il/data/images/3470_933b6ec9bbada499dc75f672964312f9.jpg" width=600 /> <img src="https://i0.wp.com/kenderter.eu/wp-content/uploads/2019/08/20190803_191408_fb.jpg" width=300 />

> MNPR01_201703 contains variant calls for all included samples and types (genomic, transcriptomic) aligned to the MNPR01_reference_201703 table. Samples can be found in the sample_info table. The MNPR01_201703 table is exported using the Google Genomics BigQuery variants schema. This table is useful for general analysis of the Cannabis genome. 
[Data Card](https://www.kaggle.com/datasets/bigquery/genomics-cannabis/data)

In [10]:
# API request - fetch the table with its reference
MNPR_table = client.get_table(
    dataset_ref.table("MNPR01_201703"))

# Preview the first five lines of the table
df=client.list_rows(MNPR_table, max_results=5).to_dataframe()
df.head()

,reference_name,start,end,reference_bases,alternate_bases,variant_id,quality,filter,names,call,...,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR,TYPE
0,gi|1098496801|gb|MNPR01000114.1|,318648,318654,TCAAAG,[TCGAAG],CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY4MDF8Z2J8TU5QUj...,24.77870,[],[],"[{'call_set_id': '17527604790083478309-1647', ...",...,0.0,[0],[1],[0],[5.18177],[1],0,0.0,0,[snp]
1,gi|1098489786|gb|MNPR01004871.1|,11908,11914,CCTTGC,[TTTTGC],CKXG8eKP9qOf8wESIGdpfDEwOTg0ODk3ODZ8Z2J8TU5QUj...,17.87680,[],[],"[{'call_set_id': '17527604790083478309-1532', ...",...,0.0,[0],[1],[1],[5.18177],[0],0,0.0,0,[mnp]
2,gi|1098496792|gb|MNPR01000118.1|,348937,348947,CTCAACTTGC,[TTCAATTTGT],CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY3OTJ8Z2J8TU5QUj...,29.52050,[],[],"[{'call_set_id': '17527604790083478309-1532', ...",...,0.0,[0],[1],[1],[5.18177],[0],0,0.0,0,[complex]
3,gi|1098495754|gb|MNPR01000694.1|,72508,72512,GTAG,[ATAG],CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU3NTR8Z2J8TU5QUj...,3.77924,[],[],"[{'call_set_id': '17527604790083478309-1532', ...",...,0.0,[1],[1],[0],[5.18177],[1],0,0.0,0,[snp]
4,gi|1098489564|gb|MNPR01005041.1|,8695,8699,TCAA,[TAAA],CKXG8eKP9qOf8wESIGdpfDEwOTg0ODk1NjR8Z2J8TU5QUj...,24.89040,[],[],"[{'call_set_id': '17527604790083478309-1532', ...",...,0.0,[0],[1],[0],[5.18177],[1],0,0.0,0,[snp]


In [11]:
MNPR_table

Table(TableReference(DatasetReference('bigquery-public-data', 'genomics_cannabis'), 'MNPR01_201703'))

Detailed explanation for the columns is hidden bellow

reference_name: The name of the reference sequence (e.g., a chromosome or a contig- partially overlapping sequence used for alignment).

start: The starting position of the variant on the reference sequence.

end: The ending position of the variant on the reference sequence.

reference_bases: The bases that are found in the reference sequence at the variant position.

alternate_bases: The alternate bases that replace the reference bases at the variant position.

variant_id: A unique identifier for the variant.

quality: The quality score of the variant call, which indicates the confidence in the variant.

filter: The filter status of the variant, indicating if it passed the variant caller's filters.

names: Names associated with the variant (e.g., dbSNP IDs).

call: Information about the variant call, typically including genotype data.

AB: Allele balance for each sample.

ABP: Allele balance probability.

AC: Allele count in genotypes, for each ALT allele.

AF: Allele frequency for each ALT allele.

AN: Total number of alleles in called genotypes.

AO: Alternate allele observation count.

CIGAR: CIGAR string representing alignment operations for the variant.

DP: Read depth, or the number of reads that cover the variant position.

DPB: Read depth per allele.

DPRA: Allele-specific read depth.

EPP: Phred-scaled p-value for strand bias.

EPPR: End position probability for each read.

GTI: Genotype quality.

LEN: Length of the variant.

MEANALT: Mean alternate allele count.

MQM: Mean mapping quality for the reference allele.

MQMR: Mean mapping quality for the alternate allele.

NS: Number of samples with data.

NUMALT: Number of alternate alleles.

ODDS: Phred-scaled probability of a sequencing error.

PAIRED: Number of reads that are properly paired.

PAIREDR: Number of reads that are improperly paired.

PAO: Number of alternate allele observations in each sample.

PQA: Phred-scaled p-value for alternate allele frequency.

PQR: Phred-scaled p-value for reference allele frequency.

PRO: Number of reads supporting the reference allele.

QA: Quality score for the alternate allele.

QR: Quality score for the reference allele.

RO: Number of reads supporting the reference allele.

RPL: Number of reads on the left side of the variant.

RPP: Phred-scaled p-value for strand bias.

RPPR: End position probability for each read on the left.

RPR: Number of reads on the right side of the variant.

RUN: Length of the longest homopolymer run of the variant allele.

SAF: Phred-scaled p-value for strand bias.

SAP: Phred-scaled p-value for strand bias.

SAR: Phred-scaled p-value for strand bias.

SRF: Number of reads supporting the reference allele on the forward strand.

SRP: Phred-scaled p-value for strand bias.

SRR: Number of reads supporting the reference allele on the reverse strand.

TYPE: Type of the variant (e.g., snp, mnp, ins, del, complex).

In [12]:
# Query to get a row count for each unique TYPE, handling array values
query = """
        SELECT type_element AS variant_type, COUNT(1) AS counts
        FROM `bigquery-public-data.genomics_cannabis.MNPR01_201703`, 
        UNNEST(TYPE) AS type_element
        GROUP BY type_element
        ORDER BY counts DESC
        """
# Execute the query and get the results in a DataFrame
type_df=client.query(query, safe_config).to_dataframe()
type_df

,variant_type,counts
0,snp,55503197
1,complex,11322366
2,mnp,4225066
3,ins,2789747
4,del,2057163


The genetic variant types which are found in this DNA sequencing data:
- snp (Single Nucleotide Polymorphism): A variation at a single position in a DNA sequence among individuals.
- complex: Variants that are more complex and don't fit neatly into other categories.
- mnp (Multiple Nucleotide Polymorphism): A variation involving multiple nucleotides in a sequence.
- ins (Insertion): An insertion of one or more nucleotides into the DNA sequence.
- del (Deletion): A deletion of one or more nucleotides from the DNA sequence.

The frequent type here is snp.

In [13]:
colors=sns.color_palette("Spectral",5).as_hex()[::-1]
fig = px.bar(type_df, x='variant_type', y='counts', title="Distribution of variant type",
             color_discrete_sequence=colors)
fig.show()

Read depth refers to the number of times a specific base (nucleotide) in the DNA is read during the sequencing process. It nakes sense that the deeper the read- the better its quality.

Investigate the correlation between read depth (DP) and variant quality

In [14]:
query = """
        SELECT AVG(quality) AS avg_quality, AVG(DP) AS avg_depth
        FROM `bigquery-public-data.genomics_cannabis.MNPR01_201703`
        GROUP BY DP
        ORDER BY avg_depth DESC
        """
# Execute the query and get the results in a DataFrame
corr=client.query(query, safe_config).to_dataframe()
corr

,avg_quality,avg_depth
0,1.026980e+07,383772.0
1,9.295360e+06,377853.0
2,1.615280e+06,282436.0
3,6.522860e+06,266459.0
4,6.996160e+06,261362.0
...,...,...
6026,3.297720e+01,5.0
6027,3.328500e+01,4.0
6028,3.137998e+01,3.0
6029,3.640627e+01,2.0


In [15]:
px.scatter(corr,x='avg_depth',y='avg_quality',marginal_y="rug",marginal_x="rug",width=600, height=600,title='Read Depth - Quality Correlation',trendline='lowess')

The transcriptome is the full range of messenger RNA, or mRNA, molecules expressed by an organism.



In [16]:
# API request - fetch the table with its reference
MNPR_transcrip_table = client.get_table(
    dataset_ref.table("MNPR01_transcriptome_201703"))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
# Preview the first five lines of the table
client.list_rows(MNPR_transcrip_table, max_results=5).to_dataframe()

,reference_name,start,end,reference_bases,alternate_bases,variant_id,quality,filter,names,call,...,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR,TYPE
0,gi|1098495686|gb|MNPR01000760.1|,103923,103934,GAAAAAAAAAC,[GAAAGAAAAC],CJfqxrb_2MW3uAESIGdpfDEwOTg0OTU2ODZ8Z2J8TU5QUj...,4.57990,[],[],"[{'call_set_id': '13289866073488864535-17', 'c...",...,0.0,[1],[1],[1],[5.18177],[0],0,0.0,0,[complex]
1,gi|1098496486|gb|MNPR01000284.1|,266295,266300,TAATC,[AAATC],CJfqxrb_2MW3uAESIGdpfDEwOTg0OTY0ODZ8Z2J8TU5QUj...,25.49130,[],[],"[{'call_set_id': '13289866073488864535-4', 'ca...",...,0.0,[0],[1],[1],[5.18177],[0],0,0.0,0,[snp]
2,gi|1098495157|gb|MNPR01001282.1|,35606,35610,GCAA,[ACAA],CJfqxrb_2MW3uAESIGdpfDEwOTg0OTUxNTd8Z2J8TU5QUj...,25.42080,[],[],"[{'call_set_id': '13289866073488864535-2', 'ca...",...,0.0,[0],[1],[0],[5.18177],[1],0,0.0,0,[snp]
3,gi|1098495602|gb|MNPR01000843.1|,83202,83206,GATT,[GATC],CJfqxrb_2MW3uAESIGdpfDEwOTg0OTU2MDJ8Z2J8TU5QUj...,6.91081,[],[],"[{'call_set_id': '13289866073488864535-4', 'ca...",...,0.0,[1],[1],[0],[5.18177],[1],0,0.0,0,[snp]
4,gi|1098491989|gb|MNPR01003513.1|,12753,12758,TACCT,[TCTCT],CJfqxrb_2MW3uAESIGdpfDEwOTg0OTE5ODl8Z2J8TU5QUj...,27.27870,[],[],"[{'call_set_id': '13289866073488864535-0', 'ca...",...,0.0,[0],[1],[0],[5.18177],[1],0,0.0,0,[mnp]


a GFF (General Feature Format) file loaded into a BigQuery table. GFF is a standard format used to describe genes and other features of DNA, RNA, and protein sequences. 

id: Unique identifier for each feature.

seq_id: The sequence identifier (e.g., chromosome or scaffold).

source: The source of the feature annotation (e.g., RefSeq, Ensembl).

type: The type of feature (e.g., region, gene, mRNA).

start: The start position of the feature.

end: The end position of the feature.

geometry: Geometric representation of the feature, often in a format suitable for spatial analysis.

score: A score associated with the feature.

strand: The strand of the feature (+ for positive, - for negative).

phase: The reading frame of the feature, relevant for CDS features.

attributes: Additional information about the feature, such as gene ID, gene name, etc.

derived_features: Possibly derived or secondary features related to the primary feature.

child_features: Features that are part of or related to the primary feature.

_part: Internal field, possibly for partitioning or internal indexing.

Check the rest of the tables

In [17]:
# API request - fetch the table with its reference
gff_table = client.get_table(
    dataset_ref.table("cs10_gff"))

# Preview the first five lines of the table
client.list_rows(gff_table, max_results=5).to_dataframe()

,id,seq_id,source,type,start,end,geometry,score,strand,phase,attributes,derived_features,child_features,_part
0,NW_022060457.1:1..117294,NW_022060457.1,RefSeq,region,1,117294,"LINESTRING(1e-07 0, 0.0117294 0)",None,+,NaN,"{'ID': ['NW_022060457.1:1..117294'], 'Name': [...",[],[],140
1,NW_022060484.1:1..74638,NW_022060484.1,RefSeq,region,1,74638,"LINESTRING(1e-07 0, 0.0074638 0)",None,+,NaN,"{'ID': ['NW_022060484.1:1..74638'], 'Name': ['...",[],[],167
2,gene-TRNAC-GCA-11,NW_022060484.1,tRNAscan-SE,gene,64886,64956,"LINESTRING(0.0064886 0, 0.0064956 0)",None,-,NaN,"{'ID': ['gene-TRNAC-GCA-11'], 'Name': ['TRNAC-...",[],[rna-TRNAC-GCA-11],167
3,exon-TRNAC-GCA-11-1,NW_022060484.1,tRNAscan-SE,exon,64886,64956,"LINESTRING(0.0064886 0, 0.0064956 0)",None,-,NaN,"{'ID': ['exon-TRNAC-GCA-11-1'], 'Name': [], 'A...",[],[],167
4,rna-TRNAC-GCA-11,NW_022060484.1,tRNAscan-SE,tRNA,64886,64956,"LINESTRING(0.0064886 0, 0.0064956 0)",None,-,NaN,"{'ID': ['rna-TRNAC-GCA-11'], 'Name': [], 'Alia...",[],[exon-TRNAC-GCA-11-1],167


Create a DataFrame for table schema

In [18]:
def df_schema(table):
    # Extract schema details
    schema_details = []
    for schema_field in table.schema:
        schema_details.append({
            'Name': schema_field.name,
            'Type': schema_field.field_type,
            'Mode': schema_field.mode
        })

    # Convert to DataFrame
    schema_df = pd.DataFrame(schema_details)

    return schema_df

In [19]:
df_schema(gff_table)

,Name,Type,Mode
0,id,STRING,NULLABLE
1,seq_id,STRING,NULLABLE
2,source,STRING,NULLABLE
3,type,STRING,NULLABLE
4,start,INTEGER,NULLABLE
5,end,INTEGER,NULLABLE
6,geometry,GEOGRAPHY,NULLABLE
7,score,STRING,NULLABLE
8,strand,STRING,NULLABLE
9,phase,INTEGER,NULLABLE


In [20]:
# API request - fetch the table with its reference
projectInfo_table = client.get_table(
    dataset_ref.table("cs3k_project_info"))

# Preview the first five lines of the table
client.list_rows(projectInfo_table, max_results=5).to_dataframe()

,AvgSpotLen,BioSample,DATASTORE_provider,DATASTORE_region,Experiment,InsertSize,LibraryLayout,Library_Name,MBases,MBytes,...,Consent,DATASTORE_filetype,Instrument,LibrarySelection,LibrarySource,LoadDate,Organism,Platform,ReleaseDate,SRA_Study
0,NaN,RSP10411,None,None,None,0,PAIRED,RSP10411,NaN,NaN,...,None,None,None,None,GENOMIC,None,Cannabis sativa,None,None,Kannapedia
1,NaN,RSP10156,None,None,None,0,PAIRED,RSP10156,NaN,NaN,...,None,None,None,None,GENOMIC,None,Cannabis sativa,None,None,Kannapedia
2,NaN,RSP10072,None,None,None,0,PAIRED,RSP10072,NaN,NaN,...,None,None,None,None,GENOMIC,None,Cannabis sativa,None,None,Kannapedia
3,NaN,RSP10084,None,None,None,0,PAIRED,RSP10084,NaN,NaN,...,None,None,None,None,GENOMIC,None,Cannabis sativa,None,None,Kannapedia
4,NaN,RSP10103,None,None,None,0,PAIRED,RSP10103,NaN,NaN,...,None,None,None,None,GENOMIC,None,Cannabis sativa,None,None,Kannapedia


In [21]:
# API request - fetch the table with its reference
vcf_table = client.get_table(
    dataset_ref.table("cs3k_vcf_cs10_dv090"))

# Preview the first five lines of the table
client.list_rows(vcf_table, max_results=5).to_dataframe()

,reference_name,start_position,end_position,geometry,reference_bases,alternate_bases,names,quality,filter,call,_part
0,ctgX97,36876,36877,"LINESTRING(0.0036876 0, 0.0036877 0)",A,[{'alt': 'G'}],[],0.0,[RefCall],"[{'name': 'SRS266830', 'genotype': [0, 0], 'ph...",211
1,ctgX97,2530,2531,"LINESTRING(0.000253 0, 0.0002531 0)",C,[{'alt': 'G'}],[],0.0,[RefCall],"[{'name': 'SRS266830', 'genotype': [0, 0], 'ph...",211
2,ctgX97,2545,2546,"LINESTRING(0.0002545 0, 0.0002546 0)",T,[{'alt': 'A'}],[],0.0,[RefCall],"[{'name': 'SRS3616242', 'genotype': [0, 0], 'p...",211
3,ctgX97,5398,5400,"LINESTRING(0.0005398 0, 0.00054 0)",AG,[{'alt': 'A'}],[],11.4,[PASS],"[{'name': 'SRS3616245', 'genotype': [1, 1], 'p...",211
4,ctgX97,5398,5400,"LINESTRING(0.0005398 0, 0.00054 0)",AG,[{'alt': 'A'}],[],9.0,[PASS],"[{'name': 'SRS374081', 'genotype': [1, 1], 'ph...",211


In addition to fetching tables from the dataset, you can join tables, unnest attributes, and perform some basic statistical operations like calculating averages (AVG), minimums (MIN), and maximums (MAX) to derive insights from the data.

This database requires some domain knowledge, for instance, the sequencing output file "as is" contains information about the sequencing process, I can explore the database further and ignore the irrelevant columns to learn about the genes and features of cannabis.